# Import environment

In [46]:
import numpy as np 
import pandas as pd
import spacy
from gensim import corpora, models, similarities
import gensim
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
import wget
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Cleaning data

In [3]:
df = pd.read_csv('C:/Users/Admin/Desktop/Business Project/CRv2-DOCS.csv',header=None)

In [4]:
df.shape

(229, 3)

In [5]:
df.head()

,0,1,2
0,0,CRv2/Chunk1/BR_Cross-London_Rail_Link_Nov_1980...,CROSSLONDON RAIL LINK This booklet is about a...
1,1,CRv2/Chunk1/British_Railways_(London)_Bill_Lor...,British Railways London Bill Lords By Order O...
2,2,CRv2/Chunk1/Greater_London_Report_27_Nov_1974....,THE GREATER LONDON REPORT CONTENTS Railways s...
3,3,CRv2/Chunk1/House_of_Commons._Fifth_report_fro...,HOUSE OF COMMONS Fifth Report from the TRANSP...
4,4,CRv2/Chunk1/House_of_Commons._First_report_fro...,HOUSE OF COMMONS o Q Q First Report from the...


In [6]:
texts = df[2]

In [7]:
stopwords = stopwords.words("english")
new_stopwords = ["would","could","take","make","give"]
for i in new_stopwords:
    stopwords.append(i)

In [8]:
def lemmatisation(text, allowed_postages=["NOUN","ADJ","VERB","ADV"]):
    nlp = spacy.load("en_core_web_sm",disable=["parser","ner"])
    text_out=[ ]
    doc = nlp(text)
    for token in doc:
        if token.pos_ in allowed_postages:
            text_out.append(token.lemma_)
    return (text_out)

In [9]:
de = [ ]
for x in range(0,len(texts)):
    if len(texts[x]) > 1000000:
        de.append(x)
for x in de:
    print(x,len(texts[x]),sep = ":")

3:1455266
52:1593514
58:2345574
73:1888094
74:3089219
98:1708577
109:1621136
111:1405510
166:11237569
184:1127302
186:1314130
190:1239113
196:1258279
215:1080873
219:1843436
222:1282861
223:1375401


# Some Documents Exceed the Maximum Capacity of spacy library, which is 10^6. Problem solved with two loop.

In [10]:
for x in de:
    if len(texts[x]) > 1000000:
        str_surplus = texts[x][1000000:len(texts[x])]
        texts = texts.append(pd.Series(str_surplus,index=[len(texts)]))
        texts[x]=texts[x][0:1000000]
        for i in range(0,len(texts)):
            if len(texts[i]) > 1000000:
                de.append(i)

In [12]:
%%time
lemma_list=[]
for text in texts:
    lemma=lemmatisation(text)
    lemma_list.append(lemma)

Wall time: 15min 15s


# Remove stopwords and words that length less than or equal to 3

In [13]:
final=[]
for l in lemma_list:
    temp = []
    for w in l:
        if w not in stopwords and len(w)>3:
            temp.append(w)
    final.append(temp)

# Bigrams and Trigrams

In [94]:
bigrams_phrase = gensim.models.Phrases(final, min_count = 3, threshold=50)
trigram_phrase = gensim.models.Phrases(bigrams_phrase[final], threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrase)
trigram = gensim.models.phrases.Phraser(trigram_phrase)

data_bigrams = []
for doc in final:
    data_bigrams.append(bigram[doc])  

data_trigrams = []
for doc in data_bigrams:
    data_trigrams.append(trigram[bigram[doc]]) 

# Tf-Idf Model

In [99]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_trigrams)
corpus = [id2word.doc2bow(text) for text in data_trigrams]

tfidf = TfidfModel(corpus , id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] 
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i] = new_bow

# Bag of Words

In [ ]:
id2word = corpora.Dictionary(final)

corpus = []
for text in final:
    new = id2word.doc2bow(text)
    corpus.append(new)
    
print(corpus[0][0:20])

# LDA modelling

In [100]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = 20,
                                           random_state = 42,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = "auto")

Wall time: 14.2 s


# Visualisation

In [101]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="PCoA")
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.273006 -0.088823       1        1  15.455792
9     -0.112927  0.391473       2        1  11.979809
19    -0.202265 -0.078677       3        1  10.574199
0     -0.245677 -0.068125       4        1  10.217539
11    -0.226904  0.018348       5        1   9.449408
8     -0.229102 -0.084912       6        1   6.901719
1     -0.053227 -0.019426       7        1   6.817930
5     -0.189965 -0.013849       8        1   6.605076
14    -0.143239  0.152456       9        1   5.037124
13    -0.164630 -0.123489      10        1   4.523834
10    -0.040285 -0.084953      11        1   3.293639
2      0.123293  0.212490      12        1   2.866348
3     -0.042880  0.007392      13        1   2.785526
7      0.123386 -0.030963      14        1   1.256053
16     0.234591  0.016837      15        1   1.173373
12     0.267672 -0.058372      16        1   0.556613
18     0.273675 -0.012724      17        1   0.332630
17     0.300295 -0.042306      18        1   0.097391
4      0.297529 -0.047313      19        1   0.066680
15     0.303665 -0.045063      20        1   0.009317, topic_info=               Term          Freq         Total Category  logprob  loglift
649            work   9839.000000   9839.000000  Default  30.0000  30.0000
607           train  11898.000000  11898.000000  Default  29.0000  29.0000
479            rail  24127.000000  24127.000000  Default  28.0000  28.0000
611       transport  33201.000000  33201.000000  Default  27.0000  27.0000
594           think  16549.000000  16549.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
237            fare      0.001120   4486.015732  Topic20 -11.3153  -5.9217
325            last      0.001120     40.587960  Topic20 -11.3154  -1.2166
5522  right_reserve      0.001120   3885.122818  Topic20 -11.3155  -5.7782
564         station      0.001120   8515.360367  Topic20 -11.3159  -6.5632
479            rail      0.001119  24127.255654  Topic20 -11.3161  -7.6049

[1315 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
663       7  0.646615        Will
663      10  0.352242        Will
2553     13  0.973561     abolish
671      16  0.997782  absolutely
674      10  0.899347  acceptable
...     ...       ...         ...
656      13  0.012537        year
6509      2  0.888092   yesterday
6509      7  0.094445   yesterday
6509     10  0.001523   yesterday
6509     15  0.015233   yesterday

[2420 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 20, 1, 12, 9, 2, 6, 15, 14, 11, 3, 4, 8, 17, 13, 19, 18, 5, 16])